# LSGA

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import random
import os 
import pandas as pd
from collections import deque


class LSGA:
    def __init__(self, sudoku_puzzle, experiment_id, puzzle_id, run_id,output_folder, pop_size=150, elite_size=50, max_generations=10000, pc1=0.2, pc2=0.1, pm1=0.3, pm2=0.05, tournament_size=2):
        # initialize class attributes
        self.sudoku_puzzle = np.array(sudoku_puzzle)  # input sudoku puzzle
        self.experiment_id = experiment_id  # experiment identifier
        self.puzzle_id = puzzle_id  # puzzle identifier
        self.run_id = run_id  # run identifier
        self.history = {'best': [], 'worst': [], 'mean': []}  # to store fitness history
        self.pop_size = pop_size  # population size
        self.elite_size = elite_size  # elite population size
        self.max_generations = max_generations  # maximum number of generations
        self.pc1 = pc1  # individual crossover rate
        self.pc2 = pc2  # row crossover rate
        self.pm1 = pm1  # swap mutation rate
        self.pm2 = pm2  # reinitialization mutation rate
        self.tournament_size = tournament_size  # tournament size for selection
        self.population = []  # main population
        self.elite_population = deque(maxlen=elite_size)  # elite population using deque with maxlen
        self.initialize_population()  # initialize the population

        self.output_folder = output_folder  # output folder for results

        # create output folder if it doesn't exist
        if not os.path.exists(self.output_folder):
            os.makedirs(self.output_folder)

    def create_associated_matrix(self, sudoku_puzzle):
        # create a matrix indicating given numbers in the puzzle
        return np.where(sudoku_puzzle != 0, 1, 0)

    def initialize_population(self):
        # initialize the population with individuals
        print("Initializing population...")
        for _ in range(self.pop_size):
            individual = self.init_individual(self.sudoku_puzzle)  # initialize individual 
            associated_matrix = self.create_associated_matrix(self.sudoku_puzzle)  # create associated matrix
            self.population.append((individual, associated_matrix))  # append to population
        print("Population initialized.")

    def init_individual(self, sudoku_puzzle):
        # create a new individual by filling the puzzle randomly
        individual = sudoku_puzzle.copy()  # copy the original puzzle
        for i in range(9):
            self.init_row(individual, i)  # initialize each row
        return individual

    def init_row(self, individual, row_index):
        # initialize a row of the individual by filling missing numbers
        row = individual[row_index]
        missing_numbers = list(set(range(1, 10)) - set(row))  # find missing numbers
        random.shuffle(missing_numbers)  # shuffle missing numbers
        for j in range(9):
            if row[j] == 0:
                row[j] = missing_numbers.pop()  # fill missing numbers randomly

    def fitness(self, chromosome):
        # calculate the fitness of an individual
        individual, _ = chromosome 

        # calculate fitness based on the number of unique numbers in each column and sub-block
        ci = sum(len(set(individual[:, i])) != 9 for i in range(9)) 
        sj = sum(len(set(individual[i:i+3, j:j+3].flatten())) != 9 for i in range(0, 9, 3) for j in range(0, 9, 3))
        return (ci + sj)  # lower fitness is better

    def selection(self):
        # select the best individual from a random sample of the population using tournament selection
        selected = random.sample(self.population, self.tournament_size)
        return min(selected, key=self.fitness)

    def crossover(self, population):
        # perform crossover operation to generate new population
        new_population = []
        for individual in population:
            if random.random() < self.pc1:
                parent1 = individual
                parent2 = random.choice(population)
                child1, child2 = parent1[0].copy(), parent2[0].copy()  # create copies for children
                associated_matrix = parent1[1]
                for i in range(9):
                    if random.random() < self.pc2:
                        # Swap rows i between child1 and child2
                        for j in range(9):
                            if associated_matrix[i][j] == 0:  # Only swap if it's not a given number
                                child1[i][j], child2[i][j] = child2[i][j], child1[i][j]

                new_population.append((child1, associated_matrix))
                new_population.append((child2, associated_matrix))
            else:
                new_population.append(individual)
        return new_population

    def mutation(self, chromosome):
        # perform mutation operations on the individual
        individual, associated_matrix = chromosome
        for i in range(9):
            # Swap mutation
            if random.random() < self.pm1:
                non_given_indices = [j for j in range(9) if associated_matrix[i, j] == 0]
                if len(non_given_indices) >= 2:
                    idx1, idx2 = random.sample(non_given_indices, 2)
                    individual[i][idx1], individual[i][idx2] = individual[i][idx2], individual[i][idx1]

            # Reinitialize mutation
            if random.random() < self.pm2:
                row = individual[i]
                non_given_indices = [j for j in range(9) if associated_matrix[i, j] == 0]
                values = [row[j] for j in non_given_indices]
                random.shuffle(values)  # shuffle values
                for j, value in zip(non_given_indices, values):
                    row[j] = value

        return individual, associated_matrix

    def column_local_search(self, chromosome):
        # perform column-based local search to remove duplicates in columns
        individual, associated_matrix = chromosome
        repeat_marks = self.mark_repeats(individual)  # mark repeated numbers in columns
        C = self.get_illegal_columns(repeat_marks)  # get illegal columns

        illegal_columns = list(C)  # convert set to list
        for i in range(len(illegal_columns)):
            for j in range(i + 1, len(illegal_columns)):
                col1, col2 = illegal_columns[i], illegal_columns[j]
                for row in range(9):
                    # check if both columns have repeated numbers in the same row
                    if repeat_marks[row][col1] == 1 and repeat_marks[row][col2] == 1:
                        # ensure the positions are not given numbers
                        if associated_matrix[row][col1] == 0 and associated_matrix[row][col2] == 0:
                            # swap the values in the row
                            individual[row][col1], individual[row][col2] = individual[row][col2], individual[row][col1]
        return individual, associated_matrix

    def sub_block_local_search(self, chromosome):
        # perform sub-block-based local search to remove duplicates in sub-blocks
        individual, associated_matrix = chromosome
        repeat_marks = self.mark_sub_block_repeats(individual)  # mark repeated numbers in sub-blocks
        S = self.get_illegal_sub_blocks(repeat_marks)  # get illegal sub-blocks

        illegal_sub_blocks = list(S)  # convert set to list
        for i in range(len(illegal_sub_blocks)):
            for j in range(i + 1, len(illegal_sub_blocks)):
                sub_block1, sub_block2 = illegal_sub_blocks[i], illegal_sub_blocks[j]
                for row in range(3):
                    row1 = sub_block1[0] * 3 + row  # calculate row index in sub-block 1
                    row2 = sub_block2[0] * 3 + row  # calculate row index in sub-block 2
                    for col in range(3):
                        col1 = sub_block1[1] * 3 + col  # calculate column index in sub-block 1
                        col2 = sub_block2[1] * 3 + col  # calculate column index in sub-block 2
                        # check if both sub-blocks have repeated numbers in the same row and column
                        if repeat_marks[row1][col1] == 1 and repeat_marks[row2][col2] == 1 and row1 == row2:
                            # ensure the positions are not given numbers
                            if associated_matrix[row1][col1] == 0 and associated_matrix[row2][col2] == 0:
                                # swap the values in the same row
                                individual[row1][col1], individual[row2][col2] = individual[row2][col2], individual[row1][col1]
        return individual, associated_matrix

    def mark_repeats(self, individual):
        # mark the repeated numbers in each column
        repeat_marks = np.zeros((9, 9))  # initialize repeat marks matrix with zeros
        for i in range(9):
            col_counts = {}  # dictionary to count occurrences in each column
            for j in range(9):
                num = individual[j][i]
                if num in col_counts:
                    col_counts[num] += 1
                else:
                    col_counts[num] = 1
            for j in range(9):
                if col_counts[individual[j][i]] > 1:  # mark repeated numbers
                    repeat_marks[j][i] = 1
        return repeat_marks

    def mark_sub_block_repeats(self, individual):
        # mark the repeated numbers in each sub-block
        repeat_marks = np.zeros((9, 9))  # initialize repeat marks matrix with zeros
        for i in range(0, 9, 3):
            for j in range(0, 9, 3):
                sub_block_counts = {}  # dictionary to count occurrences in each sub-block
                for k in range(3):
                    for l in range(3):
                        num = individual[i+k][j+l]
                        if num in sub_block_counts:
                            sub_block_counts[num] += 1
                        else:
                            sub_block_counts[num] = 1
                for k in range(3):
                    for l in range(3):
                        if sub_block_counts[individual[i+k][j+l]] > 1:  # mark repeated numbers
                            repeat_marks[i+k][j+l] = 1
        return repeat_marks

    def get_illegal_columns(self, repeat_marks):
        # get columns that contain duplicates
        illegal_columns = set()  # initialize set for illegal columns
        for j in range(9):
            if sum(repeat_marks[:, j]) > 0:  # check if there are any repeated numbers in the column
                illegal_columns.add(j)
        return illegal_columns

    def get_illegal_sub_blocks(self, repeat_marks):
        # get sub-blocks that contain duplicates
        illegal_sub_blocks = set()  # initialize set for illegal sub-blocks
        for i in range(0, 9, 3):
            for j in range(0, 9, 3):
                if np.sum(repeat_marks[i:i+3, j:j+3]) > 0:  # check if there are any repeated numbers in the sub-block
                    illegal_sub_blocks.add((i // 3, j // 3))
        return illegal_sub_blocks

    def elite_population_learning(self, new_population):
        # perform elite population learning to maintain diversity and avoid local optima
        worst_individual = sorted(new_population, key=self.fitness)[-1]  # find the worst individual

        # calculate Maxfx
        max_fitness = self.fitness(worst_individual)

        # calculate Pb
        if self.elite_population:
            random_elite = random.choice(self.elite_population)
            fitness_random = self.fitness(random_elite)
            Pb = (max_fitness - fitness_random) / max_fitness if max_fitness != 0 else 0
        else:
            Pb = 0

        # decide whether to replace the worst individual with a random elite or reinitialize
        if random.random() < Pb and self.elite_population:
            new_population[-1] = random.choice(self.elite_population)
        else:
            new_individual = self.init_individual(self.sudoku_puzzle)
            associated_matrix = self.create_associated_matrix(self.sudoku_puzzle)
            new_population[-1] = (new_individual, associated_matrix)
        
        # update the population to maintain pop_size limit
        self.population = sorted(new_population, key=self.fitness)[:self.pop_size]

    def run(self):
        # main function to run the LSGA
        print("Starting LSGA...")
        generation = 1
        best_individual = min(self.population, key=self.fitness)
        best_fitness = self.fitness(best_individual)
        total_generations_to_find_solution = 0

        # run the algorithm for max_generations or until an optimal solution is found
        while generation <= self.max_generations and best_fitness > 0: 
            new_population = [self.selection() for _ in range(self.pop_size)]
            new_population = self.crossover(new_population)
            new_population = [self.mutation(ind) for ind in new_population]
            new_population = [self.column_local_search(ind) for ind in new_population]
            new_population = [self.sub_block_local_search(ind) for ind in new_population]

            self.elite_population_learning(new_population)  # perform elite population learning

            best_individual = min(self.population, key=self.fitness)
            best_fitness = self.fitness(best_individual)

            self.elite_population.append(best_individual)  # add best individual to elite population
            if len(self.elite_population) > self.elite_size:
                self.elite_population.popleft()  # maintain elite population size
            
            # print generation and best fitness
            print(f"Generation {generation}: Best fitness = {best_fitness}")

            # store fitness values in history
            generation_best = best_fitness
            generation_worst = max(self.fitness(ind) for ind in self.population)
            generation_mean = np.mean([self.fitness(ind) for ind in self.population])
            self.history['best'].append(generation_best)
            self.history['worst'].append(generation_worst)
            self.history['mean'].append(generation_mean)

            total_generations_to_find_solution = generation

            generation += 1

            # if generation % 10 == 0:
            #     self.plot_fitness_history()

            if best_fitness == 0:
                break

        print(f"Best solution found with fitness = {best_fitness}")
        self.plot_fitness_history()
        self.plot_sudoku_solution(best_individual[0])
        
        # store history in a DataFrame
        history_df = pd.DataFrame(self.history)

        # save history to Excel file
        output_file = os.path.join(self.output_folder, f'fitness_history_{self.puzzle_id}_{self.run_id}.xlsx')
        history_df.to_excel(output_file, index=False)
 
        return best_individual[0], best_fitness, total_generations_to_find_solution
    
    def plot_fitness_history(self):
        # plot fitness history over generations
        generations = range(1,len(self.history['best'])+1)
        plt.figure(figsize=(6, 3))
        plt.plot(generations, self.history['best'], label='Best Fitness')
        plt.plot(generations, self.history['mean'], label='Mean Fitness')
        plt.plot(generations, self.history['worst'], label='Worst Fitness')
        plt.xlabel('Generation')
        plt.ylabel('Fitness')
        plt.title(f'Fitness over Generations (Puzzle {self.puzzle_id}, Run {self.run_id})')
        plt.legend()

        plt.savefig(os.path.join(self.output_folder, f'fitness_history_{self.puzzle_id}_{self.run_id}.png'))
        plt.show() 

    def plot_sudoku(self, puzzle, solution=None):
        # plot the initial puzzle and the solution
        fig, axes = plt.subplots(1, 2 if solution is not None else 1, figsize=(12, 6))
        if solution is not None:
            puzzles = [puzzle, solution]
            titles = ['Initial Puzzle', 'Solution']
        else:
            puzzles = [puzzle]
            titles = ['Puzzle']

        for ax, puzzle, title in zip(axes, puzzles, titles):
            ax.matshow(np.zeros((9, 9)), cmap='Blues', vmin=0, vmax=1)
            for i in range(9):
                for j in range(9):
                    num = puzzle[i, j]
                    if num != 0:
                        color = 'black' if self.sudoku_puzzle[i, j] != 0 else 'blue'
                        ax.text(j, i, str(num), va='center', ha='center', color=color)
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_title(title)
            for i in range(10):
                ax.axhline(i - 0.5, color='black', linewidth=2 if i % 3 == 0 else 1)
                ax.axvline(i - 0.5, color='black', linewidth=2 if i % 3 == 0 else 1)

        plt.tight_layout()
        plt.savefig(os.path.join(self.output_folder, f'sudoku_solution_{self.puzzle_id}_{self.run_id}.png'))
        plt.show()

    def plot_sudoku_solution(self, solution):
        # plot the solution to the sudoku puzzle
        self.plot_sudoku(self.sudoku_puzzle, solution)


# Effect of changing different parameters

In [ ]:
# Sample Sudoku puzzle Medium level (No.27)
sudoku_puzzle = [
    [0, 1, 0, 5, 0, 6, 0, 2, 0],
    [3, 0, 0, 0, 0, 0, 0, 0, 6],
    [0, 0, 9, 1, 0, 4, 5, 0, 0],
    [0, 9, 0, 0, 1, 0, 0, 4, 0],
    [0, 7, 0, 3, 0, 2, 0, 5, 0],
    [0, 3, 0, 0, 8, 0, 0, 6, 0],
    [0, 0, 3, 2, 0, 7, 1, 0, 0],
    [9, 0, 0, 0, 0, 0, 0, 0, 2],
    [0, 5, 0, 6, 0, 1, 0, 8, 0]
  ]

output_folder="./additional_studies_output/parameter_changing/"

#  experiment
def run_experiment(parameter, values):
    results = []

    for value in values: 
        total_generations_in_all_runs = 0
        total_best_fitness = 0
        max_run = 10  # number of runs for each parameter value

        for run_id in range(1, max_run + 1):
            print('run: ', run_id)
            # Set the default parameters
            params = {
                "pop_size": 200,
                "elite_size": 50,
                "max_generations": 500,
                "pc1": 0.8,
                "pc2": 0.75,
                "pm1": 0.3,
                "pm2": 0.055,
                "tournament_size": 2
            }
            params[parameter] = value

            # initialize LSGA with the current parameter
            lsga = LSGA(sudoku_puzzle=sudoku_puzzle, experiment_id=parameter, puzzle_id="test", run_id=run_id,
                        output_folder=output_folder+"/"+parameter+"/value_"+str(value), **params)

            # run the LSGA algorithm
            _, best_fitness, total_generations_to_find_solution = lsga.run()
 
            total_generations_in_all_runs += total_generations_to_find_solution
            total_best_fitness += best_fitness

        avg_generations = total_generations_in_all_runs / max_run
        avg_best_fitness = total_best_fitness / max_run
        results.append({'Parameter Value': value, 'Avg_Gen': avg_generations, 'Avg_Best_Fitness': avg_best_fitness})

    return results

# save results to Excel and plot them
def save_and_plot_results(parameter, results):
    df = pd.DataFrame(results)
    output_dir = output_folder
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f'results_{parameter}.xlsx')
    df.to_excel(output_file, index=False)

    # Plot results
    plt.figure(figsize=(10, 6))
    plt.plot(df['Parameter Value'], df['Avg_Gen'], label='Avg Generations')
    plt.plot(df['Parameter Value'], df['Avg_Best_Fitness'], label='Avg Best Fitness')
    plt.xlabel(parameter)
    plt.ylabel('Value')
    plt.title(f'Effect of {parameter} on LSGA Performance')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, f'plot_{parameter}.png'))
    plt.show()

# Define parameter ranges
parameter_ranges = {
    "pop_size": [50, 100, 150, 200, 250],
    "elite_size": [10, 20, 30, 40, 50], 
    "pc1": [0.2, 0.4, 0.5, 0.7, 0.9],
    "pc2": [0.2, 0.4, 0.5, 0.7, 0.9],
    "pm1": [0.1, 0.2, 0.3, 0.4, 0.5],
    "pm2": [0.01, 0.03, 0.05, 0.07, 0.1],
    "tournament_size": [2, 4, 6, 8, 10]
}

# Run experiments for all parameters
for parameter, values in parameter_ranges.items():
    print(f"Running experiment for parameter: {parameter}")
    results = run_experiment(parameter, values)
    save_and_plot_results(parameter, results)
